In [2]:
from flask import Flask, request, jsonify
import numpy as np
import joblib

In [3]:
app = Flask(__name__)
model = joblib.load('D:\Учеба\Аспирантура\Диссертация\Томск\Сериализация\catboost_IHM.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    features = np.array([data['features']])  # Предполагаем, что приходит {"features": [...]}
    probability = model.predict_proba(features)[:, 1]#[0, 1]  # Для бинарной классификации
    return jsonify({'probability': float(probability[0])})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.23.241:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jul/2025 14:49:09] "POST /predict HTTP/1.1" 200 -


In [ ]:
curl -X POST -H "Content-Type: application/json" -d "{\"features\": [81.0, 75.0, 185.0, 89.0, 0.0, 0.0, 111.0, 0.0, 0.0, 0.0, 7.0, 0.0]}"  http://localhost:5000/predict